In [1]:
import numpy as np
import pandas as pd

Features: 
* Time peak
  - Check time series distribution relative to perofrmance (pure stats) and see if there's any merit to the time of peak of a player.
* Formula (Raw Score & With Peak & Probability).
  - Come up with formula to calculate the raw score (with and without peak) and the probability of winning all-nba.
  - My theory is that we'll have to group and compare stats by position (guard, forward, and center). Then we ought to arrange all players that season and see their average standing to calculate for the weights.
  - Another theory is that the time of peak performance of a player has an effect on the likelihood that an nba player will be picked as an all-nba player.
  - Team-success should also be considered.
* Compare to average game score per season
  - Test statistical significance (Correlation or MSE) between game score and winning all-nba modell.
* Testing of accuracy
  - For the probability, we could test using MSE. With 1 being an All-NBA player and 0 if not.
  - Or arange them by their score (from derrived formula) that season and show accurate our list is based on the official all-nba. 

Problems
* Stats to consider
  - Different positions have different emphasis (guards are more considered valuable if they could facilitate while centers are more valuable for their defensive output)
  - The challenge here lies in how to get these emphasis
  - My theory is we could get this through averaging the stats of all-nba players and non all-nba players and rank them based on their deviations. The more deviation, the higher emphasis.
* Training Model for weights?
  - regression logistic
* Should we remove the classic era?
  - It is possible to identify three main Eras that we define as the Classic Era of the NBA (1980–1994), the Transitional Era of the NBA (1995–2013), and the Modern Era of the NBA (since 2013) ([link](https://content.iospress.com/articles/journal-of-sports-analytics/jsa200525))  

In [2]:
df_season_awards = pd.read_csv('/kaggle/input/nba-aba-baa-stats/End of Season Teams (Voting).csv')
df_all_nba = df_season_awards[df_season_awards['type']=='All-NBA']
df_all_nba.head(10)

,season,lg,type,number_tm,position,player,age,tm,pts_won,pts_max,share,x1st_tm,x2nd_tm,x3rd_tm,seas_id,player_id
34,2024,NBA,All-NBA,1T,C,Nikola Jokić,28,DEN,495.0,495.0,1.000,99.0,0.0,0.0,31672,4352
35,2024,NBA,All-NBA,1T,G,Shai Gilgeous-Alexander,25,OKC,495.0,495.0,1.000,99.0,0.0,0.0,31769,4670
36,2024,NBA,All-NBA,1T,G,Luka Dončić,24,DAL,493.0,495.0,0.996,98.0,1.0,0.0,31593,4654
37,2024,NBA,All-NBA,1T,F,Giannis Antetokounmpo,29,MIL,473.0,495.0,0.956,88.0,11.0,0.0,31366,4164
38,2024,NBA,All-NBA,1T,F,Jayson Tatum,25,BOS,427.0,495.0,0.863,65.0,34.0,0.0,31468,4518
39,2024,NBA,All-NBA,2T,G,Jalen Brunson,27,NYK,368.0,495.0,0.743,37.0,61.0,0.0,31429,4630
40,2024,NBA,All-NBA,2T,G,Anthony Edwards,22,MIN,285.0,495.0,0.576,3.0,87.0,9.0,31171,4808
41,2024,NBA,All-NBA,2T,F,Kevin Durant,35,PHO,274.0,495.0,0.554,2.0,85.0,9.0,31551,3770
42,2024,NBA,All-NBA,2T,F,Kawhi Leonard,32,LAC,242.0,495.0,0.489,1.0,72.0,21.0,31531,4032
43,2024,NBA,All-NBA,2T,C,Anthony Davis,30,LAL,230.0,495.0,0.465,1.0,65.0,30.0,31170,4066


In [3]:
df_season_stats = df_season_awards = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Per Game.csv')
df_season_stats[df_season_stats['player'] == 'Luka Dončić']

,seas_id,season,player_id,player,birth_year,pos,age,experience,lg,tm,...,ft_percent,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game
275,32146,2025,4653,Luka Dončić,NaN,PG,25.0,7,NBA,DAL,...,0.765,0.6,7.8,8.4,6.0,1.0,0.6,2.4,3.8,27.2
870,31593,2024,4653,Luka Dončić,NaN,PG,24.0,6,NBA,DAL,...,0.786,0.8,8.4,9.2,9.8,1.4,0.5,4.0,2.1,33.9
1570,30880,2023,4654,Luka Dončić,NaN,PG,23.0,5,NBA,DAL,...,0.742,0.8,7.8,8.6,8.0,1.4,0.5,3.6,2.5,32.4
2338,30158,2022,4654,Luka Dončić,NaN,PG,22.0,4,NBA,DAL,...,0.744,0.9,8.3,9.1,8.7,1.2,0.6,4.5,2.2,28.4
3076,29379,2021,4654,Luka Dončić,NaN,PG,21.0,3,NBA,DAL,...,0.730,0.8,7.2,8.0,8.6,1.0,0.5,4.3,2.3,27.7
3757,28704,2020,4654,Luka Dončić,NaN,PG,20.0,2,NBA,DAL,...,0.758,1.3,8.1,9.4,8.8,1.0,0.2,4.3,2.5,28.8
4425,28015,2019,4654,Luka Dončić,NaN,SG,19.0,1,NBA,DAL,...,0.713,1.2,6.6,7.8,6.0,1.1,0.3,3.4,1.9,21.2


In [4]:
print(df_season_stats.duplicated(subset=['season', 'player_id']).sum())

6053


### Combining Duplicated rows
as it turns out, players who were traded half-way have separate entry. Before we could proceed accordingly, it is imperative to combine them.

In [5]:
df_season_stats.dtypes

seas_id            int64
season             int64
player_id          int64
player            object
birth_year       float64
pos               object
age              float64
experience         int64
lg                object
tm                object
g                  int64
gs               float64
mp_per_game      float64
fg_per_game      float64
fga_per_game     float64
fg_percent       float64
x3p_per_game     float64
x3pa_per_game    float64
x3p_percent      float64
x2p_per_game     float64
x2pa_per_game    float64
x2p_percent      float64
e_fg_percent     float64
ft_per_game      float64
fta_per_game     float64
ft_percent       float64
orb_per_game     float64
drb_per_game     float64
trb_per_game     float64
ast_per_game     float64
stl_per_game     float64
blk_per_game     float64
tov_per_game     float64
pf_per_game      float64
pts_per_game     float64
dtype: object

In [6]:
# Define a function for weighted mean
def weighted_mean(series, weights):
    return (series * weights).sum() / weights.sum()

# Create an aggregation dictionary for all stats
aggregation_dict = {
    'mp_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),   # Weighted minutes
    'fg_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),   # Weighted field goals
    'fga_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted field goal attempts
    'fg_percent': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),    # Weighted FG%
    'x3p_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted 3-pointers made
    'x3pa_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']), # Weighted 3-point attempts
    'x3p_percent': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),   # Weighted 3P%
    'x2p_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted 2-pointers made
    'x2pa_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']), # Weighted 2-point attempts
    'x2p_percent': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),   # Weighted 2P%
    'e_fg_percent': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted Effective FG%
    'ft_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),   # Weighted free throws
    'fta_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted free throw attempts
    'ft_percent': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),    # Weighted FT%
    'orb_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted offensive rebounds
    'drb_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted defensive rebounds
    'trb_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted total rebounds
    'ast_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted assists
    'stl_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted steals
    'blk_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted blocks
    'tov_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted turnovers
    'pf_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),   # Weighted personal fouls
    'pts_per_game': lambda x: weighted_mean(x, df_season_stats.loc[x.index, 'g']),  # Weighted points
}

additional_aggregation = {
    'seas_id': 'first',  # Keep the first season ID
    'player': 'first',   # Keep the player's name
    'birth_year': 'first',  # Keep birth year
    'pos': 'first',  # Keep the first position
    'age': 'mean',   # Average age
    'experience': 'mean',  # Average experience
    'g': 'sum'       # Total games played
}

aggregation_dict.update(additional_aggregation)

df_combined = df_season_stats.groupby(['season', 'player_id']).agg(aggregation_dict).reset_index()
df_combined.head()

,season,player_id,mp_per_game,fg_per_game,fga_per_game,fg_percent,x3p_per_game,x3pa_per_game,x3p_percent,x2p_per_game,...,tov_per_game,pf_per_game,pts_per_game,seas_id,player,birth_year,pos,age,experience,g
0,1947,1,0.0,3.8,15.0,0.256,0.0,0.0,0.0,3.8,...,0.0,2.0,9.8,1,Al Brightman,NaN,F,23.0,1.0,58
1,1947,2,0.0,0.2,1.6,0.125,0.0,0.0,0.0,0.2,...,0.0,1.2,0.8,2,Al Lujack,NaN,F,26.0,1.0,5
2,1947,3,0.0,1.2,6.3,0.188,0.0,0.0,0.0,1.2,...,0.0,1.8,2.8,3,Al Negratti,NaN,F-C,25.0,1.0,11
3,1947,4,0.0,3.8,13.6,0.281,0.0,0.0,0.0,3.8,...,0.0,2.0,9.4,4,Angelo Musi,NaN,G,28.0,1.0,60
4,1947,5,0.0,3.8,15.7,0.241,0.0,0.0,0.0,3.8,...,0.0,3.1,9.0,5,Ariel Maughan,1923.0,F,23.0,1.0,59


In [7]:
print(df_combined.duplicated(subset=['season', 'player_id']).sum())

0


In [8]:
df_combined.to_csv('nba_per_game_no_duplicates.csv', index=False)


## Joining two DFs using 'season' and 'player_id'

In [9]:
all_nba_with_stats = df_all_nba.merge(df_combined, how='inner', on=['season', 'player_id'])
all_nba_with_stats.to_csv('all_nba_with_stats.csv', index=False)
all_nba_with_stats

,season,lg,type,number_tm,position,player_x,age_x,tm,pts_won,pts_max,...,tov_per_game,pf_per_game,pts_per_game,seas_id_y,player_y,birth_year,pos,age_y,experience,g
0,2024,NBA,All-NBA,1T,C,Nikola Jokić,28,DEN,495.0,495.0,...,3.000000,2.500000,26.40000,31672,Nikola Jokić,NaN,C,28.0,9.0,79
1,2024,NBA,All-NBA,1T,G,Shai Gilgeous-Alexander,25,OKC,495.0,495.0,...,0.610417,1.133333,4.50625,31770,Shake Milton,NaN,SG,27.0,6.0,96
2,2024,NBA,All-NBA,1T,F,Giannis Antetokounmpo,29,MIL,473.0,495.0,...,3.400000,2.900000,30.40000,31366,Giannis Antetokounmpo,NaN,PF,29.0,11.0,73
3,2024,NBA,All-NBA,1T,F,Jayson Tatum,25,BOS,427.0,495.0,...,2.500000,2.000000,26.90000,31468,Jayson Tatum,NaN,PF,25.0,7.0,74
4,2024,NBA,All-NBA,2T,G,Anthony Edwards,22,MIN,285.0,495.0,...,3.100000,1.800000,25.90000,31171,Anthony Edwards,NaN,SG,22.0,4.0,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732,1966,NBA,All-NBA,2nd,C,Bill Russell,31,BOS,5.0,9.0,...,0.000000,2.800000,12.90000,2865,Bill Russell,NaN,C,31.0,10.0,78
1733,1966,NBA,All-NBA,2nd,G,Sam Jones,32,BOS,4.0,9.0,...,0.000000,2.500000,23.50000,2964,Sam Jones,NaN,SG,32.0,9.0,67
1734,1966,NBA,All-NBA,2nd,F,John Havlicek,25,BOS,4.0,9.0,...,0.000000,2.200000,18.80000,2926,John Havlicek,NaN,SF,25.0,4.0,71
1735,1966,NBA,All-NBA,2nd,G,Hal Greer,29,PHI,2.0,9.0,...,0.000000,3.900000,22.70000,2903,Hal Greer,NaN,SG,29.0,8.0,80


### There's an error for luka doncic in the dataset (inconsistent player id)

In [10]:
df_luka = df_season_stats[df_season_stats['player'] == 'Luka Dončić']
df_luka['player_id'] = 4654
df_luka_combined = df_luka[df_luka['season'] == 2024].groupby(['season', 'player_id']).agg(aggregation_dict).reset_index()

df_combined_new = pd.concat([df_season_stats, df_luka_combined], ignore_index=True)
df_combined_new.to_csv('nba_per_game_no_duplicates_fixed.csv', index=False)

/tmp/ipykernel_17/2649868081.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_luka['player_id'] = 4654


### Loaded the saved csv here for it to be quickly loaded instead of calculating all over again

In [11]:
nba_stats_per_game = pd.read_csv('/kaggle/working/nba_per_game_no_duplicates_fixed.csv')

In [12]:
all_nba_with_stats = df_all_nba.merge(nba_stats_per_game, how='inner', on=['season', 'player_id'])
all_nba_with_stats.to_csv('all_nba_with_stats_fixed.csv', index=False)
all_nba_with_stats.head()

,season,lg_x,type,number_tm,position,player_x,age_x,tm_x,pts_won,pts_max,...,ft_percent,orb_per_game,drb_per_game,trb_per_game,ast_per_game,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game
0,2024,NBA,All-NBA,1T,C,Nikola Jokić,28,DEN,495.0,495.0,...,0.817,2.8,9.5,12.4,9.0,1.4,0.9,3.0,2.5,26.4
1,2024,NBA,All-NBA,1T,G,Shai Gilgeous-Alexander,25,OKC,495.0,495.0,...,0.816,0.3,1.3,1.6,1.3,0.4,0.1,0.6,1.1,4.5
2,2024,NBA,All-NBA,1T,G,Shai Gilgeous-Alexander,25,OKC,495.0,495.0,...,0.818,0.2,1.2,1.3,1.3,0.4,0.1,0.7,1.2,4.7
3,2024,NBA,All-NBA,1T,G,Shai Gilgeous-Alexander,25,OKC,495.0,495.0,...,0.667,1.8,2.8,4.5,1.5,0.5,0.3,0.8,0.8,6.8
4,2024,NBA,All-NBA,1T,G,Shai Gilgeous-Alexander,25,OKC,495.0,495.0,...,1.000,0.0,1.0,1.0,0.7,0.2,0.0,0.0,1.2,1.8


## Defining their eligible each persons eligible positions
I did this by combining player season info ('pos' coloumn) and the nba stats by 'player_id'.

In [13]:
df_season_info_players = pd.read_csv('/kaggle/input/nba-aba-baa-stats/Player Season Info.csv')
df_season_info_players.rename(columns={'pos': 'eligible_position'}, inplace=True)
df_season_info_players.head()

,season,seas_id,player_id,player,birth_year,eligible_position,age,lg,tm,experience
0,1947,1,1,Al Brightman,NaN,F,23.0,BAA,BOS,1
1,1947,2,2,Al Lujack,NaN,F,26.0,BAA,WSC,1
2,1947,3,3,Al Negratti,NaN,F-C,25.0,BAA,WSC,1
3,1947,4,4,Angelo Musi,NaN,G,28.0,BAA,PHW,1
4,1947,5,5,Ariel Maughan,1923.0,F,23.0,BAA,DTF,1


In [14]:
df_season_stats_new = nba_stats_per_game.merge(
    df_season_info_players[['season', 'player_id', 'eligible_position']], 
    how='left',
    on=['season', 'player_id']
)
df_season_stats_new[df_season_stats_new['season']>= 1979]['eligible_position'].unique()

array(['SG', 'PF', 'PG', 'SF', 'C', nan, 'SF-SG', 'PF-SF', 'SG-PG',
       'SG-SF', 'PG-SG', 'C-PF', 'SG-PG-SF', 'PF-C', 'SF-PF', 'SF-C',
       'SG-PF', 'PG-SF'], dtype=object)

## Calculate average stats per season and position

In [15]:
df_season_stats_new['eligible_position'] = df_season_stats_new['eligible_position'].str.strip().str.upper()
df_season_stats_new_new = df_season_stats_new.assign(
    expanded_positions=df_season_stats_new['eligible_position'].str.split('-')
)

expanded_stats = df_season_stats_new_new.explode('expanded_positions')
expanded_stats = expanded_stats.rename(columns={'expanded_positions': 'primary_pos'})

def map_primary_pos(pos):
    if isinstance(pos, str):
        if 'G' in pos:
            return 'G'
        elif 'F' in pos:
            return 'F'
        elif 'C' in pos:
            return 'C'
    return None

expanded_stats['mapped_pos'] = expanded_stats['primary_pos'].apply(map_primary_pos)

filtered_stats = expanded_stats[expanded_stats['mapped_pos'].notna()]
filtered_stats = filtered_stats[filtered_stats['season'] >= 1979]

stats_list = list(aggregation_dict.items())
numeric_columns = [x[0] for x in stats_list if x[0] != 'player' and x[0] != 'pos']
filtered_stats = filtered_stats[numeric_columns + ['season', 'mapped_pos']]

average_stats_by_season_pos = (
    filtered_stats.groupby(['season', 'mapped_pos'])[numeric_columns]
    .mean()
    .reset_index()
)

average_stats_by_season_pos

,season,mapped_pos,mp_per_game,fg_per_game,fga_per_game,fg_percent,x3p_per_game,x3pa_per_game,x3p_percent,x2p_per_game,...,stl_per_game,blk_per_game,tov_per_game,pf_per_game,pts_per_game,seas_id,birth_year,age,experience,g
0,1979,C,19.609735,3.273451,6.679646,0.453796,NaN,NaN,NaN,3.273451,...,0.504425,0.823894,1.669027,2.560177,8.185841,7832.814159,1950.000000,25.849558,4.477876,51.637168
1,1979,F,19.573953,3.725116,7.626977,0.471474,NaN,NaN,NaN,3.725116,...,0.625581,0.438140,1.634419,2.216279,9.231628,7826.651163,1951.319149,25.409302,4.246512,48.386047
2,1979,G,20.311060,3.669585,8.088018,0.436347,NaN,NaN,NaN,3.669585,...,0.918894,0.163594,1.810138,1.972811,9.014286,7803.958525,1950.093750,25.958525,4.529954,52.133641
3,1980,C,23.163366,3.772277,7.522772,0.490020,0.002970,0.025743,0.111895,3.766337,...,0.689109,0.957426,1.750495,2.711881,9.454455,8152.029703,1949.333333,26.772277,5.524752,53.574257
4,1980,F,21.678481,3.821097,8.002110,0.461039,0.032489,0.135021,0.200869,3.786498,...,0.720253,0.475105,1.726160,2.440506,9.654852,8167.025316,1951.113636,26.172996,4.734177,49.493671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,2024,F,17.674488,2.676350,5.777840,0.442581,0.893482,2.516015,0.322794,1.786592,...,0.565922,0.374860,0.787523,1.404283,7.222533,31515.307263,NaN,27.089385,6.150838,35.432030
137,2024,G,18.225586,2.870117,6.712109,0.410846,1.036523,3.023242,0.327800,1.830273,...,0.636523,0.223438,0.987695,1.304492,7.926563,31526.875000,2000.500000,26.263672,5.750000,36.625000
138,2025,C,19.177966,3.345763,6.010169,0.562316,0.450847,1.172881,0.349086,2.896610,...,0.503390,0.822034,1.133898,2.045763,8.752542,32070.440678,NaN,26.559322,6.610169,4.016949
139,2025,F,18.996774,3.080000,6.776129,0.449932,0.916774,2.721935,0.311290,2.168387,...,0.643871,0.471613,1.047742,1.740000,8.506452,32088.116129,NaN,25.593548,5.283871,3.896774


## Add Team Success 
## Calculate Deviation



## Finding Emphasis by Positions
### Grouping Players by positions (guards, forwards, centers)
### Get All-nba players per season.
#### Average their stats.
### Get Average stats by players per season.
### Calculate deviations then sort descending.
### Analyze each.

